In [1]:
import uproot
import h5py
import numpy as np
import os
import re
import awkward as awk
import heplot as hepl
import matplotlib.pyplot as plt
import importlib
import glob
from heplot import plotting as heplt
import sys
sys.path.append('..')

In [2]:
import src.selection as sele
import src.string_constants as stco
import src.generator as gene
import src.reader as read

# read all MC by dsid weighted

In [3]:
feat_in = ['el_e','mu_e','el_charge','mu_charge','el_pt','mu_pt', 'el_phi','mu_phi',\
                    'el_eta', 'mu_eta', 'jet_e', 'jet_pt', 'jet_truthflav',\
                	'weight_mc', 'weight_pileup', 'weight_jvt', 'weight_leptonSF']
feat_out = ['jet_pt_lead','jet_truthflav_lead',\
                	'weight_mc', 'weight_pileup', 'weight_jvt', 'weight_leptonSF']

In [4]:
def generate_all_evts_for_dsid(dsid, path, feat_in, feat_out, N=int(1e5)):
    
    gen = gene.sample_generator([path], N, selection_fun=sele.select_lightjets, feature_names_in=feat_in, feature_names_out=feat_out)

    samples = None
    i = 0
    
    for sample_batch in gen:
    
        if samples is None:
            samples = sample_batch
        else:
            samples = awk.concatenate([samples,sample_batch])
    
        i = i+1
        if i > 5:
            break
    
    print(f'{len(samples)} samples read for dsid {dsid}')

    return samples

In [5]:
paths = {dsid : glob.glob(os.path.join(stco.in_dir_mc,'*'+dsid+'*'))[0] for dsid in stco.ds_ids_all}

In [ ]:
sample_dd = {}

for dsid, path in paths.items():

    # read samples from all files for a dsid
    samples = generate_all_evts_for_dsid(dsid, path, feat_in, feat_out, N=int(1e2))
    # compute and apply event weights
    # add to dict
    sample_dd[dsid] = samples

600 samples read for dsid 410472
600 samples read for dsid 363356
600 samples read for dsid 363358
600 samples read for dsid 364302
600 samples read for dsid 506193
600 samples read for dsid 506194


In [8]:
paths

['/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.410472.PhPy8EG_A14_ttbar_hdamp258p75_dil.D_FTAG2.e6348_s3681_r13144_p5981.GN2v01_Nom_output.root',
 '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.363356.Sherpa_221_NNPDF30NNLO_ZqqZll.D_FTAG2.e5525_s3681_r13144_p5981.GN2v01_Nom_output.root',
 '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.363358.Sherpa_221_NNPDF30NNLO_WqqZll.D_FTAG2.e5525_s3681_r13144_p5981.GN2v01_Nom_output.root',
 '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.364302.Sherpa_222_NNPDF30NNLO_ggZllZqq.D_FTAG2.e6273_s3681_r13144_p5981.GN2v01_Nom_output.root',
 '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.506193.MGPy8EG_Zee_FxFx_3jets_HT2b

In [9]:
stco.ds_ids_all

['410472',
 '363356',
 '363358',
 '364302',
 '506193',
 '506194',
 '506195',
 '506196',
 '506197',
 '506198',
 '512198',
 '512199',
 '512200']

In [13]:
{dsid : glob.glob(os.path.join(stco.in_dir_mc,'*'+dsid+'*'))[0] for dsid in stco.ds_ids_all}

{'410472': '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.410472.PhPy8EG_A14_ttbar_hdamp258p75_dil.D_FTAG2.e6348_s3681_r13144_p5981.GN2v01_Nom_output.root',
 '363356': '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.363356.Sherpa_221_NNPDF30NNLO_ZqqZll.D_FTAG2.e5525_s3681_r13144_p5981.GN2v01_Nom_output.root',
 '363358': '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.363358.Sherpa_221_NNPDF30NNLO_WqqZll.D_FTAG2.e5525_s3681_r13144_p5981.GN2v01_Nom_output.root',
 '364302': '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.ltoffoli.mc20_13T.364302.Sherpa_222_NNPDF30NNLO_ggZllZqq.D_FTAG2.e6273_s3681_r13144_p5981.GN2v01_Nom_output.root',
 '506193': '/eos/atlas/atlascerngroupdisk/perf-flavtag/calib/negtag/Rel24_GN2v01/Ntuples_Nominal/mc20d/user.lt